#### 生成した関数をテストデータに張り付ける  
1. 変更後の関数をテストデータに貼り付け
   - f_gold -> f_filled
   - コメント文 TOFILL の下に貼り付け
   - meta式の計算機精度などを計算するテストを作成
1. 翻訳後の動的解析用のコードを作成
1. 翻訳前の動的解析用のコードを作成

In [ ]:
from lib763.fs import (
    save_str_to_file,
    load_str_from_file,
    get_all_file_names_in,
    get_all_dir_names_in,
    get_all_file_path_in,
    ensure_path_exists,
    create_zip_from_list,
    get_file_name,
    get_file_rows_iter,
    get_file_name_without_ext
)
from lib763.regex import replace_patterns, extract_matching_strings

In [ ]:
from CONST import (
    J2PY_META_TEST_DIR,
    PY2J_META_TEST_DIR,
    J2PY_GENERATED_DIR,
    PY2J_GENERATED_DIR,
    J2PY_EVAL_TEST_DIR,
    PY2J_EVAL_TEST_DIR,
    AFTER_DA_PYTHON_DIR,
    AFTER_DA_JAVA_DIR,
    PYTHON_TGT_FUNC_DIR,
    BEFORE_DA_JAVA_DIR,
    BEFORE_DA_PYTHON_DIR,
    FIXED_BEFORE_JAVA_DIR,
    FIXED_BEFORE_PYTHON_DIR,
)

In [ ]:
# meta式のテストを保存するディレクトリ
ensure_path_exists(J2PY_EVAL_TEST_DIR)
ensure_path_exists(PY2J_EVAL_TEST_DIR)

# 翻訳語のテストを保存するディレクトリ
ensure_path_exists(AFTER_DA_JAVA_DIR)
ensure_path_exists(AFTER_DA_PYTHON_DIR)

In [ ]:
test_j2py_path_ls = get_all_file_path_in(J2PY_META_TEST_DIR)
test_py2j_path_ls = get_all_file_path_in(PY2J_META_TEST_DIR)

1. 変更後の関数をテストデータに貼り付け


In [ ]:
def add_func_tofill_below(func_path, test_path):
    # 生成した関数の名前をf_filledに変更する
    func_impl_str = replace_patterns(
        load_str_from_file(func_path), {"f_gold": "f_filled"}
    )
    test_impl_str = load_str_from_file(test_path)
    # コメントの下に貼り付け
    return "\n".join(
        [
            row + (("\n" + func_impl_str) if "TOFILL" in row else "")
            for row in test_impl_str.split("\n")
        ]
    )

In [ ]:
# metaのテストを生成：python
for dir_name in get_all_dir_names_in(J2PY_GENERATED_DIR):
    save_dir = f"{J2PY_EVAL_TEST_DIR}{dir_name}/"
    ensure_path_exists(save_dir)
    tgt_func_parent_dir = f"{J2PY_GENERATED_DIR}{dir_name}/"
    tgt_test_file_path = f"{J2PY_META_TEST_DIR}{dir_name}.py"

    for file_name in get_all_file_names_in(tgt_func_parent_dir):
        tgt_func_path = f"{tgt_func_parent_dir}{file_name}"
        new_func = add_func_tofill_below(tgt_func_path, tgt_test_file_path)
        save_str_to_file(new_func, f"{save_dir}{file_name}")

In [ ]:
# pythonのpatch: numpyのインポート
for path in get_all_file_path_in(f"{J2PY_EVAL_TEST_DIR}"):
    save_str_to_file("\n".join(["import numpy as np"] + get_file_rows_iter(path)), path)

In [ ]:
# metaのテストを生成：java
for dir_name in get_all_dir_names_in(PY2J_GENERATED_DIR):
    save_dir = f"{PY2J_EVAL_TEST_DIR}{dir_name}/"
    ensure_path_exists(save_dir)
    tgt_func_parent_dir = f"{PY2J_GENERATED_DIR}{dir_name}/"
    tgt_test_file_path = f"{PY2J_META_TEST_DIR}{dir_name}.java"

    for file_name in get_all_file_names_in(tgt_func_parent_dir):
        tgt_func_path = f"{tgt_func_parent_dir}{file_name}"
        new_func = add_func_tofill_below(tgt_func_path, tgt_test_file_path)
        save_str_to_file(new_func, f"{save_dir}{file_name}")

In [ ]:
# patch for java: import javafxを削除
for path in get_all_file_path_in(f"{PY2J_EVAL_TEST_DIR}"):
    save_str_to_file(
        "\n".join(
            [
                row
                for row in get_file_rows_iter(path)
                if not ("import" in row and "javafx" in row)
            ]
        ),
        path,
    )

2. 翻訳後の関数を動的解析するためのコードを作成


python


In [ ]:
def create_after_python(test_name):
    # test_nameはテスト名(exp:ADD_1_TO_A_GIVEN_NUMBER)

    # コメント文と無駄な行を排除した行の配列
    # ソースコードはtest_by_meta
    src_row_ls = [
        row
        for row in load_str_from_file(f"{J2PY_META_TEST_DIR}{test_name}.py").split("\n")
        if (not row.replace(" ", "").startswith("#") and len(row) != 0)
    ]

    # import文の行のリスト
    import_row_ls = [row for row in src_row_ls if "import" in row]

    # paramの実装を取得
    param_row_ls = []
    flag = False
    bracket_num = 0
    param_row = []
    for row in src_row_ls:
        if not flag and "param" in row and "=" in row:
            flag = True
        if flag:
            param_row_ls.append(row.replace(" ", ""))
            bracket_num += row.count("[")
            bracket_num -= row.count("]")
            if bracket_num == 0:
                break
    try:
        exec("\n".join(param_row_ls))
        exec("param_row.append(f'f_filled(*{param[0]})')")
    except:
        print(test_name)
        return None
    if len(param_row) == 0:
        print(test_name)
        return None

    for cand_path in get_all_file_path_in(f"{J2PY_GENERATED_DIR}{test_name}"):
        cand_file_name = get_file_name(cand_path)
        # f_filled関数:transcoderにより生成された関数
        f_filled_row_ls = replace_patterns(
            load_str_from_file(cand_path), {"f_gold": "f_filled"}
        ).split("\n")

        save_str_to_file(
            "\n".join(import_row_ls + f_filled_row_ls + param_row),
            f"{AFTER_DA_PYTHON_DIR}{test_name}/{cand_file_name}",
        )

In [ ]:
for test_name in get_all_dir_names_in(J2PY_EVAL_TEST_DIR):
    ensure_path_exists(f"{AFTER_DA_PYTHON_DIR}{test_name}/")
    create_after_python(test_name)

In [ ]:
# patch1 import numpy
for path in get_all_file_path_in(AFTER_DA_PYTHON_DIR):
    save_str_to_file("\n".join(["import numpy as np"] + get_file_rows_iter(path)), path)

In [ ]:
# patch2
for path in get_all_file_path_in(FIXED_BEFORE_PYTHON_DIR):
    testname = get_file_name_without_ext(path)
    for row in get_file_rows_iter(path)[::-1]:
        if 'f_gold' in row:
            invoke_row = row
            break
    invoke_row = invoke_row.replace('f_gold','f_filled')
    for testpath in get_all_file_path_in(f'{AFTER_DA_PYTHON_DIR}{testname}/'):
        save_row_ls = get_file_rows_iter(testpath)[::-1]
        for i,row in enumerate(save_row_ls.copy()):
            if 'f_filled' in row:
                save_row_ls[i] = invoke_row
                break
        save_str_to_file('\n'.join(save_row_ls[::-1]),testpath)

java


In [ ]:
def create_after_java(test_name):
    # test_nameはテスト名(exp:ADD_1_TO_A_GIVEN_NUMBER)

    # コメント文と無駄な行を排除した行の配列
    # ソースコードはtest_by_meta
    src_row_ls = [
        row
        for row in load_str_from_file(f"{PY2J_META_TEST_DIR}{test_name}.java").split(
            "\n"
        )
        if (not row.replace(" ", "").startswith("//") and len(row) != 0)
    ]
    src_row_ls = [
        row for row in src_row_ls if not ("javafx" in row and "import" in row)
    ]

    # f_goldを削除する
    removed_fgold_row_ls = []
    finished = False
    fgold_flag = False
    bracket_num = 0
    for row in src_row_ls:
        if finished:
            removed_fgold_row_ls.append(row)
            continue
        if not fgold_flag and not "f_gold" in row:
            removed_fgold_row_ls.append(row)
            continue
        else:
            fgold_flag = True
        # f_gold関数の行を対処する
        if fgold_flag:
            bracket_num += row.count("{")
            bracket_num -= row.count("}")
            if bracket_num == 0:
                finished = True
                fgold_flag = False

    # main文を変更する
    main_changed_str_ls = []
    main_flag = False
    bracket_num = 1
    param_dict = {}
    file_str_ls = removed_fgold_row_ls.copy()
    for i, row in enumerate(removed_fgold_row_ls):
        if not main_flag:
            # not in main method
            main_changed_str_ls.append(row)
            main_flag = "void main" in row
            continue
        # main method
        bracket_num += row.count("{")
        bracket_num -= row.count("}")
        if bracket_num == 0:
            # end of main method
            main_flag = False
            main_changed_str_ls.append(
                "f_filled(" + ",".join(param_dict.values()) + ");"
            )
            main_changed_str_ls.append("}")
        else:
            # still in main method
            if "param" in row and "add" in row:
                param_num = row.split(".")[0]
                if not param_num in param_dict.keys():
                    param_all_row_ls = []
                    for param_row in file_str_ls[i:]:
                        param_all_row_ls.append(param_row.replace("  ", " "))
                        if ";" in param_row:
                            break
                    param_str = extract_matching_strings(
                        "".join(param_all_row_ls), r"\((.*?)\)"
                    )[0]
                    param_dict[param_num] = param_str
    main_start_index = next(
        (
            idx
            for idx, row in enumerate(main_changed_str_ls)
            if "main" in row and "static" in row
        ),
        None,
    )
    if main_start_index is None:
        print(test_name)
        return
    for cand_path in get_all_file_path_in(f"{PY2J_GENERATED_DIR}{test_name}"):
        cand_file_name = get_file_name(cand_path)
        # f_filled関数:transcoderにより生成された関数
        f_filled_row_ls = replace_patterns(
            load_str_from_file(cand_path), {"f_gold": "f_filled"}
        ).split("\n")

        save_str = "\n".join(
            main_changed_str_ls[:main_start_index]
            + f_filled_row_ls
            + main_changed_str_ls[main_start_index:]
        )

        save_str_to_file(
            save_str,
            f"{AFTER_DA_JAVA_DIR}{test_name}/{cand_file_name}",
        )

In [ ]:
for test_name in get_all_dir_names_in(PY2J_EVAL_TEST_DIR):
    ensure_path_exists(f"{AFTER_DA_JAVA_DIR}{test_name}/")
    create_after_java(test_name)

3. 翻訳前の動的解析用コードを作成

python

In [ ]:
def create_before_python(test_name: str):
    # 翻訳前のコード：f_gold
    f_gold_row_ls = load_str_from_file(f"{PYTHON_TGT_FUNC_DIR}{test_name}.py").split(
        "\n"
    )

    # コメント文と無駄な行を排除した行の配列
    # ソースコードはtest_by_meta
    src_row_ls = [
        row
        for row in load_str_from_file(f"{J2PY_META_TEST_DIR}{test_name}.py").split("\n")
        if (not row.replace(" ", "").startswith("#") and len(row) != 0)
    ]

    # import文の行のリスト
    import_row_ls = [row for row in src_row_ls if "import" in row]

    # paramの実装を取得
    param_row_ls = []
    flag = False
    bracket_num = 0
    param_row = []
    for row in src_row_ls:
        if not flag and "param" in row and "=" in row:
            flag = True
        if flag:
            param_row_ls.append(row.replace(" ", ""))
            bracket_num += row.count("[")
            bracket_num -= row.count("]")
            if bracket_num == 0:
                break
    try:
        exec("\n".join(param_row_ls))
        exec("param_row.append(f'f_gold(*{param[0]})')")
    except:
        print(test_name)
        return
    if len(param_row) == 0:
        print(test_name)
        return

    save_str_to_file(
        "\n".join(import_row_ls + f_gold_row_ls + param_row),
        f"{BEFORE_DA_PYTHON_DIR}{test_name}.py",
    )

In [ ]:
ensure_path_exists(f"{BEFORE_DA_PYTHON_DIR}")
for test_name in get_all_dir_names_in(J2PY_EVAL_TEST_DIR):
    create_before_python(test_name)

java

In [ ]:
def create_before_java(test_name):
    # test_nameはテスト名(exp:ADD_1_TO_A_GIVEN_NUMBER)

    # コメント文と無駄な行を排除した行の配列
    # ソースコードはtest_by_meta
    src_row_ls = [
        row
        for row in load_str_from_file(f"{PY2J_META_TEST_DIR}{test_name}.java").split(
            "\n"
        )
        if (not row.replace(" ", "").startswith("//") and len(row) != 0)
    ]
    src_row_ls = [
        row for row in src_row_ls if not ("javafx" in row and "import" in row)
    ]

    # main文を変更する
    main_changed_str_ls = []
    main_flag = False
    bracket_num = 1
    param_dict = {}
    for i, row in enumerate(src_row_ls):
        if not main_flag:
            # not in main method
            main_changed_str_ls.append(row)
            main_flag = "void main" in row
            continue
        # main method
        bracket_num += row.count("{")
        bracket_num -= row.count("}")
        if bracket_num == 0:
            # end of main method
            main_flag = False
            main_changed_str_ls.append("f_gold(" + ",".join(param_dict.values()) + ");")
            main_changed_str_ls.append("}")
        else:
            # still in main method
            if "param" in row and "add" in row:
                param_num = row.split(".")[0]
                if not param_num in param_dict.keys():
                    param_all_row_ls = []
                    for param_row in src_row_ls[i:]:
                        param_all_row_ls.append(param_row.replace("  ", " "))
                        if ";" in param_row:
                            break
                    param_str = extract_matching_strings(
                        "".join(param_all_row_ls), r"\((.*?)\)"
                    )[0]
                    param_dict[param_num] = param_str

    save_str_to_file(
        "\n".join(main_changed_str_ls),
        f"{BEFORE_DA_JAVA_DIR}{test_name}.java",
    )

In [ ]:
ensure_path_exists(BEFORE_DA_JAVA_DIR)
for test_name in get_all_dir_names_in(PY2J_EVAL_TEST_DIR):
    create_before_java(test_name)

create zip

In [ ]:
save_str_to_file("", "input.txt")
for test_name in get_all_dir_names_in(PY2J_EVAL_TEST_DIR):
    file_path_ls = (
        get_all_file_path_in(f"{AFTER_DA_PYTHON_DIR}{test_name}")
        + get_all_file_path_in(f"{AFTER_DA_JAVA_DIR}{test_name}")
        + [
            "./input.txt",
            f"{BEFORE_DA_JAVA_DIR}{test_name}.java",
            f"{BEFORE_DA_PYTHON_DIR}{test_name}.py",
        ]
    )
    create_zip_from_list(file_path_ls, f"{test_name}.zip")